<a href="https://colab.research.google.com/github/fabiansep/TalentOps/blob/main/m2_E5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

# Crear dataset comprehensivo de e-commerce
np.random.seed(42)
n_pedidos = 2500

# Generar fechas
fechas = pd.date_range('2023-01-01', periods=n_pedidos, freq='H')[:n_pedidos]

# Crear datos base
df = pd.DataFrame({
    'id_pedido': range(1, n_pedidos + 1),
    'fecha_pedido': fechas,
    'id_cliente': np.random.randint(1, 501, n_pedidos),
    'categoria': np.random.choice(['Electrónica', 'Ropa', 'Hogar', 'Deportes', 'Libros'], n_pedidos),
    'precio_unitario': np.round(np.random.uniform(10, 1000, n_pedidos), 2),
    'cantidad': np.random.randint(1, 5, n_pedidos),
    'metodo_pago': np.random.choice(['Tarjeta', 'PayPal', 'Efectivo', 'Transferencia'], n_pedidos, p=[0.6, 0.2, 0.15, 0.05]),
    'region': np.random.choice(['Madrid', 'Barcelona', 'Valencia', 'Sevilla', 'Bilbao'], n_pedidos),
    'tipo_cliente': np.random.choice(['Regular', 'Premium', 'VIP'], n_pedidos, p=[0.7, 0.2, 0.1])
})

# Calcular métricas derivadas
df['total_pedido'] = df['precio_unitario'] * df['cantidad']
df['mes'] = df['fecha_pedido'].dt.month
df['dia_semana'] = df['fecha_pedido'].dt.day_name()

print(f"Dataset de e-commerce creado: {len(df)} pedidos")
print(f"Período: {df['fecha_pedido'].min()} a {df['fecha_pedido'].max()}")

Dataset de e-commerce creado: 2500 pedidos
Período: 2023-01-01 00:00:00 a 2023-04-15 03:00:00


/tmp/ipython-input-2767170003.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  fechas = pd.date_range('2023-01-01', periods=n_pedidos, freq='H')[:n_pedidos]


In [2]:
# Análisis de calidad de datos
print("ANÁLISIS DE CALIDAD DE DATOS")
print("=" * 30)
print(f"Dimensiones: {df.shape}")
print(f"Tipos de datos:\n{df.dtypes}")
print(f"Valores faltantes: {df.isnull().sum().sum()}")

# Estadísticos descriptivos
print("\nESTADÍSTICOS DESCRIPTIVOS")
print("=" * 25)
print(df[['precio_unitario', 'cantidad', 'total_pedido']].describe())

# Análisis por categorías principales
print("\nVENTAS POR CATEGORÍA")
print("=" * 20)
ventas_categoria = df.groupby('categoria').agg({
    'total_pedido': ['count', 'sum', 'mean'],
    'cantidad': 'sum'
}).round(2)
print(ventas_categoria)

# Análisis temporal
print("\nVENTAS POR MES")
print("=" * 15)
ventas_mes = df.groupby('mes').agg({
    'total_pedido': 'sum',
    'id_pedido': 'count'
}).round(2)
print(ventas_mes)

# Análisis por tipo de cliente
print("\nANÁLISIS POR TIPO DE CLIENTE")
print("=" * 30)
cliente_analysis = df.groupby('tipo_cliente').agg({
    'total_pedido': ['mean', 'sum', 'count'],
    'cantidad': 'mean'
}).round(2)
print(cliente_analysis)

ANÁLISIS DE CALIDAD DE DATOS
Dimensiones: (2500, 12)
Tipos de datos:
id_pedido                   int64
fecha_pedido       datetime64[ns]
id_cliente                  int64
categoria                  object
precio_unitario           float64
cantidad                    int64
metodo_pago                object
region                     object
tipo_cliente               object
total_pedido              float64
mes                         int32
dia_semana                 object
dtype: object
Valores faltantes: 0

ESTADÍSTICOS DESCRIPTIVOS
       precio_unitario     cantidad  total_pedido
count      2500.000000  2500.000000   2500.000000
mean        499.551080     2.474800   1249.215124
std         284.437272     1.109711    965.553384
min          10.030000     1.000000     14.150000
25%         252.047500     2.000000    482.487500
50%         493.130000     2.000000    965.520000
75%         744.240000     3.000000   1840.942500
max         999.560000     4.000000   3991.080000

VENTAS POR

In [3]:
# Convertir variables categóricas para correlación
df_corr = df.copy()
df_corr['tipo_cliente_num'] = df_corr['tipo_cliente'].map({'Regular': 1, 'Premium': 2, 'VIP': 3})

# Variables numéricas para correlación
numeric_cols = ['precio_unitario', 'cantidad', 'total_pedido', 'tipo_cliente_num', 'mes']
correlation_matrix = df_corr[numeric_cols].corr()

print("\nMATRIZ DE CORRELACIÓN")
print("=" * 20)
print(correlation_matrix.round(3))

# Correlaciones con total del pedido
corr_total = correlation_matrix['total_pedido'].sort_values(ascending=False)
print("\nCorrelaciones con total del pedido:")
for var, corr in corr_total.items():
    if var != 'total_pedido':
        print(f"{var:15} | {corr:+.3f}")


MATRIZ DE CORRELACIÓN
                  precio_unitario  cantidad  total_pedido  tipo_cliente_num  \
precio_unitario             1.000     0.041         0.743            -0.007   
cantidad                    0.041     1.000         0.613            -0.017   
total_pedido                0.743     0.613         1.000            -0.018   
tipo_cliente_num           -0.007    -0.017        -0.018             1.000   
mes                        -0.005     0.002        -0.003             0.002   

                    mes  
precio_unitario  -0.005  
cantidad          0.002  
total_pedido     -0.003  
tipo_cliente_num  0.002  
mes               1.000  

Correlaciones con total del pedido:
precio_unitario | +0.743
cantidad        | +0.613
mes             | -0.003
tipo_cliente_num | -0.018


In [4]:
# Outliers en precios
Q1_precio = df['precio_unitario'].quantile(0.25)
Q3_precio = df['precio_unitario'].quantile(0.75)
IQR_precio = Q3_precio - Q1_precio

outliers_precio = df[df['precio_unitario'] > Q3_precio + 1.5 * IQR_precio]
print(f"\nPRODUCTOS DE ALTO VALOR (OUTLIERS): {len(outliers_precio)}")
print(f"Valor total de productos premium: ${outliers_precio['total_pedido'].sum():,.2f}")

# Análisis por día de la semana
ventas_dia = df.groupby('dia_semana')['total_pedido'].agg(['count', 'sum', 'mean']).round(2)
print("\nVENTAS POR DÍA DE LA SEMANA")
print("=" * 30)
print(ventas_dia.sort_values('sum', ascending=False))


PRODUCTOS DE ALTO VALOR (OUTLIERS): 0
Valor total de productos premium: $0.00

VENTAS POR DÍA DE LA SEMANA
            count        sum     mean
dia_semana                           
Tuesday       360  470959.22  1308.22
Monday        360  452611.76  1257.25
Sunday        360  450868.25  1252.41
Friday        360  449411.17  1248.36
Wednesday     360  438142.36  1217.06
Saturday      340  433770.68  1275.80
Thursday      360  427274.37  1186.87


In [5]:
# Outliers en precios
Q1_precio = df['precio_unitario'].quantile(0.25)
Q3_precio = df['precio_unitario'].quantile(0.75)
IQR_precio = Q3_precio - Q1_precio

outliers_precio = df[df['precio_unitario'] > Q3_precio + 1.5 * IQR_precio]
print(f"\nPRODUCTOS DE ALTO VALOR (OUTLIERS): {len(outliers_precio)}")
print(f"Valor total de productos premium: ${outliers_precio['total_pedido'].sum():,.2f}")

# Análisis por día de la semana
ventas_dia = df.groupby('dia_semana')['total_pedido'].agg(['count', 'sum', 'mean']).round(2)
print("\nVENTAS POR DÍA DE LA SEMANA")
print("=" * 30)
print(ventas_dia.sort_values('sum', ascending=False))


PRODUCTOS DE ALTO VALOR (OUTLIERS): 0
Valor total de productos premium: $0.00

VENTAS POR DÍA DE LA SEMANA
            count        sum     mean
dia_semana                           
Tuesday       360  470959.22  1308.22
Monday        360  452611.76  1257.25
Sunday        360  450868.25  1252.41
Friday        360  449411.17  1248.36
Wednesday     360  438142.36  1217.06
Saturday      340  433770.68  1275.80
Thursday      360  427274.37  1186.87
